In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Analysis of World Happiness Report

**Introduction** - World Happiness Report is ranked among 156 Countries based on a Cantril Ladder Survey. Nationally representative samples of respondents are asked to think of a ladder, the best possible life for them being a 10, and the worst possible experience is a 0. They are then asked to rate their own current lives on that 0 to 10 scale.

I always Wondered what are the Metrics that are used to define a Happiness Score for a country.

Dataset is taken from Kaggle's Website

For this project, I was interestested in Analysing World Happiness Report and understand few findings from the Dataset:

**1.Which Region Has Highest Happiest Countries?**

**2.Will Trust in Government Show Great Impact on Happiness Score?**

**3.Trend of All the Features for Different Countries Over Time**

# Import Libraries Needed

In [ ]:
#To Handle Dataset
import pandas as pd
import numpy as np
from collections import defaultdict

#To make plots and Visualize the Data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

#TO Build a ML Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error


Most important step before we start working on the data is **Assess** the Raw Data and **Clean** the Data
In this Report we have 6 different Excels with each excel contains the Report from 2015 to 2020. 
Interesting part is :
**1.** Not all the Excels have same column Names 
**2.** 2020 Report has Few Extra Columns which the other Datasets doesn't Have

In [ ]:
#Read all the Excel Files which have Report Data from 2016 to 2020

##########     Reading the 2020 Report ##########################
df_2020=pd.read_csv('../input/world-happiness-report/2020.csv')

#There is no Happiness Rank in the Datset while Remaining Datasets Have. good part is data is ordered as per the Rank
df_2020['Happiness Rank'] =  range(1, len(df_2020.index)+1)

#Renaming the Columns so that all datasets have similar Header Names which will be Required later to Analyse Data from cumulative Years

df_2020 = df_2020.rename(columns = {'Country name' : 'Country', 'Ladder score' : 'Happiness Score', 
                        'Logged GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family', 
                                    'Healthy life expectancy' : 'Health (Life Expectancy)',
                        'Freedom to make life choices' : 'Freedom', 
                                    'Perceptions of corruption' : 'Trust (Government Corruption)'})

#Adding Year Column in order to differentiate data easily
df_2020['Year']=2020

##########     Reading the 2019 Report ##########################
df_2019 = pd.read_csv('../input/world-happiness-report/2019.csv')
df_2019['Year']=2019
df_2019 = df_2019.rename(columns = {'Overall rank':'Happiness Rank', 'Country or region' : 'Country', 'Score' : 'Happiness Score',
                                      'GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family',
                                      'Healthy life expectancy' : 'Health (Life Expectancy)','Freedom to make life choices' : 'Freedom'
                                     , 'Perceptions of corruption' : 'Trust (Government Corruption)'})


##########     Reading the 2018 Report ##########################
df_2018 = pd.read_csv('../input/world-happiness-report/2018.csv')
df_2018['Year']=2018
df_2018 = df_2018.rename(columns = {'Overall rank':'Happiness Rank', 'Country or region' : 'Country', 'Score' : 'Happiness Score',
                                      'GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family',
                                      'Healthy life expectancy' : 'Health (Life Expectancy)',
                                      'Freedom to make life choices' : 'Freedom',
                                      'Perceptions of corruption' : 'Trust (Government Corruption)'})

##########     Reading the 2017 Report ##########################
df_2017 = pd.read_csv('../input/world-happiness-report/2017.csv')
df_2017 = df_2017.rename(columns = {'Happiness.Rank':'Happiness Rank', 'Happiness.Score' : 'Happiness Score', 
                                      'Economy..GDP.per.Capita.' : 'Economy (GDP per Capita)', 'Health..Life.Expectancy.' : 'Health (Life Expectancy)',
                                      'Trust..Government.Corruption.' : 'Trust (Government Corruption)', 'Dystopia.Residual' : 'Dystopia Residual'})
df_2017['Year']=2017

##########     Reading the 2016 Report ##########################
df_2016= pd.read_csv('../input/world-happiness-report/2016.csv')
df_2016['Year']=2016

##########     Reading the 2015 Report ##########################
df_2015 = pd.read_csv('../input/world-happiness-report/2015.csv')
df_2015['Year']=2015

##########  Concat all the Years Data into a single dataframe ##########################
df_all = pd.concat([df_2020,df_2019,df_2018,df_2017,df_2016,df_2015])

df_all=df_all[['Country', 'Happiness Rank', 'Happiness Score', 'Economy (GDP per Capita)',
                                   'Family', 'Health (Life Expectancy)', 'Freedom','Trust (Government Corruption)',
                                   'Generosity', 'Year']]
#Print Top 5 rows from the dataset
df_all.head()

In [ ]:
df_2020.info()

In [ ]:
df_2020.describe()

# Which Region Has Highest Happiest Countries?

In [ ]:
## Visualising Regions with Top Happiness Score 
plt.figure(figsize=(10,5))
df_2020.head(20).groupby('Regional indicator').agg({'Country':'count'}).sort_values(by='Country',ascending=False).plot(kind='bar',color='g',title='')
plt.show()

## Visualising Regions with Least Happiness Score 
plt.figure(figsize=(10,5))
df_2020.tail(20).groupby('Regional indicator').agg({'Country':'count'}).sort_values(by='Country',ascending=True).plot(kind='bar',color='r')
plt.show()

**Conclusion** : Western Europe Region have Highestwith highest Happiness Score While Latin Americs and Carribean with the Least

## Histogram shows the Variablity of Each Metric

In [ ]:
plt.rcParams['figure.figsize'] = (16,16)
## Visulaize the Variability of Each Metric using Histogram plot.
df_2020[['Happiness Score',
        'upperwhisker', 'lowerwhisker',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Generosity', 'Trust (Government Corruption)',
        'Explained by: Log GDP per capita',
       'Explained by: Social support', 'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual']].hist();

df_2020.columns

****Happiness Score comparison of Top 10 countries****

In [ ]:
#Visualizing the Top 10 Happiest Countries using Pie Chart

top10_countryname = df_2020["Country"].iloc[0:10]
top10_countryscore = df_2020["Happiness Score"].iloc[0:10]
fig1, ax1 = plt.subplots(figsize=(10,6))
ax1.pie(top10_countryscore, explode = (0,0,0,0,0,0,0,0,0,0.1), labels=top10_countryname, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("The score comparison of Top 10 countries")
plt.show()

## Least 10 Countries by Overall Happiness Score

In [ ]:
df_top20 = df_2020[-10:].sort_values('Happiness Score', ascending = True)
px.bar(df_top20, x='Happiness Score', y='Country',orientation='h',title="Least 10 happiest countries")

# What is the Correlation Between Each Metric

In [ ]:
#Corelation of each Variable with each other
plt.figure(figsize=(10,7))
col=df_all[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity','Happiness Score']].corr()
sns.heatmap(col,annot=True,fmt='.2f');
plt.title("Correlation Map from Past 5 years")

col=df_2020[['Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual','Happiness Score']].corr()
plt.figure(figsize=(10,7))
sns.heatmap(col,annot=True)
plt.title('Correlattion Map for 2020 Dataset')
plt.show()

**Conclusion** : Happiness Score is correlated with GDP,Health expectancy and Social support and is not corellated to Generosity for 2020 Dataset

Happiness Score is correlated with Family,Freedom to Make choices and is not corellated to Generosity for the Data from Past 5 years(2016-2020)

**Filling Missing Values with Mean of each column**

Filling Missing Values with Mean of Each Column. Here Mean Method is choosen because Each row is a country and every country has a different score for each metric so mode can't be choosen. That's why Mean Method will take the Average of the entire column and fill null values with it which is Reasonable Replacement

In [ ]:
#FIll missing Values with Mean
df_all.fillna((df_all.mean()), inplace = True)
df_2020.fillna((df_2020.mean()), inplace = True)

Plot and Visulaize Top 10 Countries for Each Feature

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2,constrained_layout=True,figsize=(10,10))
sns.barplot(x='Economy (GDP per Capita)',y='Country',
                        data=df_all.nlargest(10,'Economy (GDP per Capita)'),
                        ax=axes[0,0],palette=None)
sns.barplot(x='Health (Life Expectancy)' ,y='Country',
                        data=df_all.nlargest(10,'Health (Life Expectancy)'),
                        ax=axes[0,1],palette=None)
sns.barplot(x='Happiness Score' ,y='Country',
                        data=df_all.nlargest(10,'Happiness Score'),
                        ax=axes[1,0],palette=None)
sns.barplot(x='Generosity' ,y='Country',
                        data=df_all.nlargest(10,'Generosity'),
                        ax=axes[1,1],palette=None)
sns.barplot(x='Freedom' ,y='Country',
                        data=df_all.nlargest(10,'Freedom'),
                        ax=axes[2,0],palette=None)
sns.barplot(x='Trust (Government Corruption)' ,y='Country',
                        data=df_all.nlargest(10,'Trust (Government Corruption)'),
                        ax=axes[2,1],palette=None)

### Trend of Different Features over the Years against the Happiness Score - Analysed for Few Countries

In [ ]:
countries=['India','United States','United Kingdom','Russia','China','Canada','Germany','France','Switzerland', 'Iceland', 'Denmark', 'Norway', 'Finland',
       'Netherlands','Japan', 'South Korea','Italy']
df_i=df_all[df_all['Country'].isin(countries)]
fig=px.line(df_i,x='Year',y='Happiness Score',color='Country',title='Countries Trend with Happiness Score')
fig.show()

fig=px.line(df_i,x='Year',y='Economy (GDP per Capita)',color='Country',title='Countries Trend with Economy (GDP per Capita)')
fig.show()

fig=px.line(df_i,x='Year',y='Trust (Government Corruption)',color='Country',title='Countries Trend with Trust (Government Corruption)')
fig.show()

# Designing Linear Regression Model to Predict Happiness Score

In [ ]:
def lmmodel(df):
    '''
    INPUT - DataFrame
    OUTPUT - Returns 
    r2 score for Test Dataset 
    Length of Test Datset
    r2 score for Train Dataset
    Length of Train Dataset
    '''
    #Choosing X and Y columns Y- Happiness Score which needs to be Predicted X - Features to Tarin Model
    y=df['Happiness Score']
    X=df[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)','Freedom', 'Generosity',
           'Trust (Government Corruption)']]
    
    #Splitting Test and Train Dataset
    X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=0.2,random_state=42)
    
    #Initiating Linear Regression Model
    lin_reg = LinearRegression()
    
    #Fit the Model
    lin_reg.fit(X_train,Y_train)
    
    #Predict the Happiness Score for Test and Train Dataset
    y_test_preds = lin_reg.predict(X_test)
    y_train_preds = lin_reg.predict(X_train)
    
    #Finding Score, Mean Squared Error and Mean Absolute Error
    score=lin_reg.score(X_test, Y_test)
    mse = mean_squared_error(Y_test, y_test_preds)
    mae = mean_absolute_error(Y_test, y_test_preds)
    
    #R2 Score for Model
    r2_test = r2_score(Y_test, y_test_preds)
    r2_train = r2_score(Y_train, y_train_preds) 
    
    #Length of Test and Train Dataset
    len_ytest = len(y_test_preds)
    len_ytrain = len(y_train_preds)
    
    return r2_test, len_ytest, r2_train, len_ytrain

    

In [ ]:
#Predicting Happiness Score for 2020 Dataset by using lmmodel Function
r2_test, len_ytest, r2_train,len_ytrain = lmmodel(df_2020)
print("The r-squared score for the Test model using only quantitative variables was {} on {} values.".format(r2_test,len_ytest))
print("The r-squared score for the Test model using only quantitative variables was {} on {} values.".format(r2_train,len_ytrain))
#Predicting Happiness Score for total Dataset by using lmmodel Function

r2_test, len_ytest, r2_train,len_ytrain = lmmodel(df_all)
print("The r-squared score for the Train model using only quantitative variables was {} on {} values.".format(r2_test,len_ytest))
print("The r-squared score for the Train model using only quantitative variables was {} on {} values.".format(r2_train,len_ytrain))

# Will Trust in Government Show Great Impact on Happiness Score?

In [ ]:
fig = plt.figure(figsize = (16, 8))
(ax1, ax2,ax3), (ax4,ax5, ax6) = fig.subplots(2,3)

#Plotting Trust in Government FOr Top 10 Happiest Countries over the Years
ax1.set_title("Trust in the government 2015")
df_all[df_all['Year'] == 2015][['Country','Trust (Government Corruption)']][:10].set_index('Country').plot(kind='bar', ax=ax1, legend=False)

ax2.set_title("Trust in the government 2016")
df_all[df_all['Year'] == 2016][['Country','Trust (Government Corruption)']][:10].set_index('Country').plot(kind='bar', ax=ax2, legend=False)

ax3.set_title("Trust in the government 2017")
df_all[df_all['Year'] == 2017][['Country','Trust (Government Corruption)']][:10].set_index('Country').plot(kind='bar', ax=ax3, legend=False)

ax4.set_title("Trust in the government 2018")
df_all[df_all['Year'] == 2018][['Country','Trust (Government Corruption)']][:10].set_index('Country').plot(kind='bar', ax=ax4, legend=False)

ax5.set_title("Trust in the government 2019")
df_all[df_all['Year'] == 2019][['Country','Trust (Government Corruption)']][:10].set_index('Country').plot(kind='bar', ax=ax5, legend=False)

ax6.set_title("Trust in the government 2020")
df_all[df_all['Year'] == 2020][['Country','Trust (Government Corruption)']][:10].set_index('Country').plot(kind='bar', ax=ax6, legend=False)

plt.tight_layout(pad=0.3)

**Conclusion** - Trust in Government has been drstically decresed in Year 2020 compared to past 5 years.While Iceland has their Trust in government increasing over years

Below Conclusions are drawn based on the observations we got from the data

1.Happiest Countries have Higher GDP Per Capita, Social Support from Family, Trust in Government, More Life Expectancy and 
people have more Freedom to make Life choices compared to Unhappiest Countries

2.Generosity doesn’t impact the Happiness Score of a country. Least Happy Countries are more generous

3. Finland ranked 1st for the past 3 Years with the Highest Happiness Score of 7.5

4. Some of the Countries have seen an increase in 2020 but many countries faced a downfall of the Happiness score never seen in 5 years.

5.Trust in the government and support of family has fallen over the years.

6.There is a vast difference in the economic state of countries, which may be a big factor impacting the happiness of the countries.
